# Neural Network with tensorflow on MNIST

In [29]:
import tensorflow as tf
import numpy as np
# import matplotlib.pyplot as plt
import gzip, pickle

## MNIST

In [30]:
# Import MNIST data

with gzip.open('mnist.pkl.gz', 'rb') as f:
    ((traind,trainl),(vald,vall),(testd,testl))=pickle.load(f, encoding='bytes')
    traind = traind.astype("float32").reshape(-1,28,28)
    trainl = trainl.astype("float32")
    testd = testd.astype("float32").reshape(-1,28,28)
    testl = testl.astype("float32")

# traind = traind.reshape(-1,784) 
testd = testd.reshape(-1,784)

print("train images: ", traind.shape)
print("train labels: ", trainl.shape)

print("test images: ", testd.shape)
print("test labels: ", testl.shape)

train images:  (60000, 28, 28)
train labels:  (60000, 10)
test images:  (10000, 784)
test labels:  (10000, 10)


In [31]:
# split MNIST

# classes 0-4
classVector = trainl.argmax(axis=1)
class01234Mask = np.logical_or((classVector == 0), 
                               np.logical_or((classVector == 1), 
                               np.logical_or((classVector == 2), 
                               np.logical_or((classVector == 3), 
                                             (classVector == 4)))))

trainl_01234 = trainl[class01234Mask]
traind_01234 = traind[class01234Mask]

traind_01234 = traind_01234.reshape(-1,784)

print("classes 0-4, images: ", trainl_01234.shape)
print("classes 0-4, labels: ", traind_01234.shape)


# classes 5-9
class56789Mask = np.logical_or((classVector == 5), 
                               np.logical_or((classVector == 6), 
                               np.logical_or((classVector == 7), 
                               np.logical_or((classVector == 8), 
                                             (classVector == 9)))))

trainl_56789 = trainl[class56789Mask]
traind_56789 = traind[class56789Mask]

traind_56789 = traind_56789.reshape(-1,784)

print("classes 5-9, images: ", trainl_56789.shape)
print("classes 5-9, labels: ", traind_56789.shape)


classes 0-4, images:  (30596, 10)
classes 0-4, labels:  (30596, 784)
classes 5-9, images:  (29404, 10)
classes 5-9, labels:  (29404, 784)


In [32]:
# batch creation

def get_batch(data, labels, batch_size, index, epoch):
    
    # if size of traind exceeded, restart!
    if index >= (data.shape[0] // batch_size):
        index = 0
        epoch += 1
    
    # draw batches
    dataBatch = data[index * batch_size:(index+1) * batch_size]
    labelBatch = labels[index * batch_size:(index+1) * batch_size]
    index += 1
    
    return dataBatch, labelBatch, index, epoch

## Neural Network

In [33]:
# Parameters
learning_rate = 0.001
training_iters = 2000
batch_size = 128
display_step = 100

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_hidden_1 = 200 # 1st layer number of neurons
n_hidden_2 = 200 # 2st layer number of neurons
n_hidden_3 = 200 # 3st layer number of neurons
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])

In [34]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='wh1'),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='wh2'),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]), name='wh3'),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]), name='wo')
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='bh1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='bh2'),
    'b3': tf.Variable(tf.random_normal([n_hidden_3]), name='bh3'),
    'out': tf.Variable(tf.random_normal([n_classes]), name='bo')
}

In [35]:
def neural_network(x, w, b):
    # Hidden fully connected layer with 200 neurons
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, w['h1']), b['b1']))
    
    # Hidden fully connected layer with 200 neurons
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, w['h2']), b['b2']))
    
    # Hidden fully connected layer with 200 neurons
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, w['h3']), b['b3']))
    
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_3, w['out']) + b['out']
    
    return out_layer

In [36]:
# Construct model
logits = neural_network(X, weights, biases)
# prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
update = optimizer.minimize(loss_op)
# print(optimizer.compute_gradients(loss_op))

# Evaluate model
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## Training

In [37]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
# Start training
sess = tf.Session()
# Run the initializer
sess.run(init)

In [38]:
# train classes 0-4

iteration = 0
epoch = 0
for step in range(0, training_iters):
    batch_x, batch_y, iteration, epoch = get_batch(traind_01234, trainl_01234, batch_size, iteration, epoch)

    # Run optimization op (backprop)
    sess.run(update, feed_dict={X: batch_x, Y: batch_y})

    if step % display_step == 0 or step == 1:
        # Calculate batch loss and accuracy
        loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})

        print("Step " + str(step) + \
              ", Epoch: " + str(epoch) + \
              ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))


Step 0, Epoch: 0, Minibatch Loss= 22747.2266, Training Accuracy= 0.234
Step 1, Epoch: 0, Minibatch Loss= 26368.9648, Training Accuracy= 0.484
Step 100, Epoch: 0, Minibatch Loss= 123.1363, Training Accuracy= 0.945
Step 200, Epoch: 0, Minibatch Loss= 60.0898, Training Accuracy= 0.953
Step 300, Epoch: 1, Minibatch Loss= 55.1753, Training Accuracy= 0.953
Step 400, Epoch: 1, Minibatch Loss= 25.5581, Training Accuracy= 0.977
Step 500, Epoch: 2, Minibatch Loss= 16.7239, Training Accuracy= 0.992
Step 600, Epoch: 2, Minibatch Loss= 29.8613, Training Accuracy= 0.969
Step 700, Epoch: 2, Minibatch Loss= 23.9550, Training Accuracy= 0.977
Step 800, Epoch: 3, Minibatch Loss= 26.1366, Training Accuracy= 0.961
Step 900, Epoch: 3, Minibatch Loss= 2.0685, Training Accuracy= 0.992
Step 1000, Epoch: 4, Minibatch Loss= 2.9668, Training Accuracy= 0.992
Step 1100, Epoch: 4, Minibatch Loss= 3.6881, Training Accuracy= 0.992
Step 1200, Epoch: 5, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 1300, Epoch: 

In [39]:
# show accuracy
print("Testing Accuracy:", \
    sess.run(accuracy, feed_dict={X: testd,
                                      Y: testl}))

Testing Accuracy: 0.4942


In [40]:
# train classes 5-9

iteration = 0
epoch = 0
for step in range(0, training_iters):
    batch_x, batch_y, iteration, epoch = get_batch(traind_56789, trainl_56789, batch_size, iteration, epoch)

    # Run optimization op (backprop)
    sess.run(update, feed_dict={X: batch_x, Y: batch_y})

    if step % display_step == 0 or step == 1:
        # Calculate batch loss and accuracy
        loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})

        print("Step " + str(step) + \
              ", Epoch: " + str(epoch) + \
              ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))


Step 0, Epoch: 0, Minibatch Loss= 2061.1802, Training Accuracy= 0.180
Step 1, Epoch: 0, Minibatch Loss= 1146.1792, Training Accuracy= 0.406
Step 100, Epoch: 0, Minibatch Loss= 142.4723, Training Accuracy= 0.891
Step 200, Epoch: 0, Minibatch Loss= 39.8204, Training Accuracy= 0.906
Step 300, Epoch: 1, Minibatch Loss= 24.3947, Training Accuracy= 0.953
Step 400, Epoch: 1, Minibatch Loss= 55.2888, Training Accuracy= 0.914
Step 500, Epoch: 2, Minibatch Loss= 47.3211, Training Accuracy= 0.930
Step 600, Epoch: 2, Minibatch Loss= 23.4015, Training Accuracy= 0.969
Step 700, Epoch: 3, Minibatch Loss= 29.6355, Training Accuracy= 0.961
Step 800, Epoch: 3, Minibatch Loss= 7.5735, Training Accuracy= 0.961
Step 900, Epoch: 3, Minibatch Loss= 2.5853, Training Accuracy= 0.969
Step 1000, Epoch: 4, Minibatch Loss= 31.3688, Training Accuracy= 0.953
Step 1100, Epoch: 4, Minibatch Loss= 1.1159, Training Accuracy= 0.992
Step 1200, Epoch: 5, Minibatch Loss= 21.0131, Training Accuracy= 0.961
Step 1300, Epoch: 5

In [41]:
# show accuracy
print("Testing Accuracy:", \
    sess.run(accuracy, feed_dict={X: testd,
                                      Y: testl}))

Testing Accuracy: 0.4588
